In [ ]:
import io
import pandas as pd
import polars as pl
import msoffcrypto
from openpyxl import load_workbook

# Configurar 
pl.set_random_seed(42)

# CONFIGURACIÓ
ruta_excel = r'ruta/del/arxiu.xlsx'
tu_contraseña = '*****'

# Desxifrar arxiu
decrypted_workbook = io.BytesIO()
with open(ruta_excel, 'rb') as file:
    office_file = msoffcrypto.OfficeFile(file)
    office_file.load_key(password=tu_contraseña)
    office_file.decrypt(decrypted_workbook)

# Obtindre noms dels fulls
decrypted_workbook.seek(0)
wb = load_workbook(decrypted_workbook, read_only=True)
nombres_hojas = wb.sheetnames
print("Fulls trobats:")
for i, nombre in enumerate(nombres_hojas):
    print(f"{i + 1}: {nombre}")

# Llegir només el full número 5
indice_hoja = 5  # Full 5 
nombre_hoja5 = nombres_hojas[indice_hoja]

decrypted_workbook.seek(0)
df_hoja5 = pd.read_excel(decrypted_workbook, sheet_name=nombre_hoja5, engine='openpyxl')
df = df_hoja5

# Mostrar primeres files del full 5
print(f"\nPrimeres files del full 5: {nombre_hoja5}")
print(df_hoja5.head())

# Preparar el DataFrame
if isinstance(df, pl.DataFrame):
    df_pl = df
else:
    df_pl = pl.from_pandas(df)

print(f"Dataset original: {df_pl.shape}")

# Definir estratègia de fusió per a cada variable 
def crear_columna_fusionada(df, columnas_candidatas, nombre_resultado):
    """Crea una columna fusionada prenent el primer valor no-null de les candidates - DETERMINISTA"""
    columnas_existentes = [col for col in columnas_candidatas if col in df.columns]

    if not columnas_existentes:
        return df.with_columns(pl.lit(None).alias(nombre_resultado))

    # Ordenar columnes alfabèticament 
    columnas_existentes = sorted(columnas_existentes)
    
    # Utilitzar coalesce per a prendre el primer valor no-null
    expresion = pl.coalesce([pl.col(col) for col in columnas_existentes])
    return df.with_columns(expresion.alias(nombre_resultado))

# Fusionar totes les variables duplicades
print("\nFUSIONANT VARIABLES DUPLICADES:")

# Fusionar Albúmina
df_fusionado = crear_columna_fusionada(
    df_pl,
    ['ALBÚMINA', 'resultadoALBÚMINA'],
    'ALBUMINA_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['fechaResultado_ALBUMINA', 'fechaResultadoALBÚMINA'],
    'FECHA_ALBUMINA_FUSIONADA'
)

# Fusionar PCR
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['PCR', 'resultadoPCR'],
    'PCR_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['fechaResultadoPCR', 'fechaResultadoPCR1'],
    'FECHA_PCR_FUSIONADA'
)

# Fusionar Creatinina
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['resultadoCREATININA', 'resultadoCREATININA1'],
    'CREATININA_FUSIONADA'
)

# Per a creatinina només hi ha una data
df_fusionado = df_fusionado.with_columns(
    pl.col('fechaResultadoCREATININA').alias('FECHA_CREATININA_FUSIONADA')
)

# Fusionar dades demogràfiques duplicades
df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descTipoPaciente', 'descTipoPaciente1'],
    'TIPO_PACIENTE_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descLinea', 'descLinea1'],
    'LINEA_FUSIONADA'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descEsquema', 'descEsquema1'],
    'ESQUEMA_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['codigoPostal', 'codigoPostal1'],
    'CODIGO_POSTAL_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descHospitalCrc', 'descHospitalCrc1'],
    'HOSPITAL_FUSIONADO'
)

df_fusionado = crear_columna_fusionada(
    df_fusionado,
    ['descZona', 'descZona1'],
    'ZONA_FUSIONADA'
)

# Verificar quantas dades recuperem
print("DADES RECUPERADES:")
print(f"Albúmina fusionada: {df_fusionado['ALBUMINA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"PCR fusionada: {df_fusionado['PCR_FUSIONADA'].drop_nulls().len():,} valors")
print(f"Creatinina fusionada: {df_fusionado['CREATININA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"Data Albúmina fusionada: {df_fusionado['FECHA_ALBUMINA_FUSIONADA'].drop_nulls().len():,} valors")
print(f"Data PCR fusionada: {df_fusionado['FECHA_PCR_FUSIONADA'].drop_nulls().len():,} valors")

# Crear el mapatge per a la transformació
variables_fusionadas = {
    'Rockwood': {'valor': 'Rockwood', 'fecha': 'F_Rockwood'},
    'ALBUMINA': {'valor': 'ALBUMINA_FUSIONADA', 'fecha': 'FECHA_ALBUMINA_FUSIONADA'},
    'PCR': {'valor': 'PCR_FUSIONADA', 'fecha': 'FECHA_PCR_FUSIONADA'},
    'CREATININA': {'valor': 'CREATININA_FUSIONADA', 'fecha': 'FECHA_CREATININA_FUSIONADA'},
    'EUROQOL': {'valor': 'descResultadoEscala1', 'fecha': 'fechaEUROQOL'}
}

# Columnes demogràfiques fusionades
cols_demograficas = ['SIPCOD', 'sexo', 'fecha Naci', 'fecha Exitus',
                     'TIPO_PACIENTE_FUSIONADO', 'LINEA_FUSIONADA', 'ESQUEMA_FUSIONADO',
                     'CODIGO_POSTAL_FUSIONADO', 'HOSPITAL_FUSIONADO', 'ZONA_FUSIONADA']

print(f"\nPREPARATS PER A TRANSFORMAR AMB TOTES LES VARIANTS FUSIONADES")
print(f"Variables a processar: {list(variables_fusionadas.keys())}")

# Transformació a format llarg amb dades fusionades
print("\nTRANSFORMANT A FORMAT LLARG:")

dfs_lista = []

# Processar variables simples (no EuroQol)
variables_ordenadas = sorted([var for var in variables_fusionadas.keys() if var != 'EUROQOL'])

for variable in variables_ordenadas:
    config = variables_fusionadas[variable]
    col_valor = config['valor']
    col_fecha = config['fecha']

    print(f"Processant {variable}...")

    # Filtrar només registres amb dades vàlides
    df_temp = df_fusionado.filter(
        (pl.col(col_valor).is_not_null()) &
        (pl.col(col_fecha).is_not_null())
    ).select([
        pl.col('SIPCOD').cast(pl.Utf8),
        pl.col(col_fecha).cast(pl.Utf8).alias('Fecha'),
        pl.col(col_valor).cast(pl.Utf8).alias('Valor'),
        *[pl.col(col).cast(pl.Utf8) for col in sorted(cols_demograficas[1:])]  # ORDE DETERMINISTA
    ]).with_columns(
        pl.lit(variable).alias('Variable')
    )

    df_temp = df_temp.sort(['SIPCOD', 'Fecha', 'Variable'])
    
    dfs_lista.append(df_temp)
    print(f"  {variable}: {len(df_temp):,} registres vàlids")

# Unir variables simples
if dfs_lista:
    df_simple_fusionado = pl.concat(dfs_lista, how="vertical")
    df_simple_fusionado = df_simple_fusionado.sort(['SIPCOD', 'Fecha', 'Variable'])
    print(f"\nVariables simples unides: {len(df_simple_fusionado):,} files")
else:
    df_simple_fusionado = pl.DataFrame()

# Processar EuroQol
print("\nProcessant EuroQol...")
euroqol_config = variables_fusionadas['EUROQOL']

euroqol_df = df_fusionado.filter(
    (pl.col(euroqol_config['valor']).is_not_null()) &
    (pl.col(euroqol_config['fecha']).is_not_null())
).select([
    pl.col('SIPCOD').cast(pl.Utf8),
    pl.col(euroqol_config['fecha']).cast(pl.Utf8).alias('Fecha'),
    pl.col(euroqol_config['valor']).cast(pl.Utf8).alias('Valor_Original'),
    *[pl.col(col).cast(pl.Utf8) for col in sorted(cols_demograficas[1:])] 
])

euroqol_df = euroqol_df.sort(['SIPCOD', 'Fecha'])

print(f"Dataset EuroQol: {len(euroqol_df):,} files")

# Expandir EuroQol 
if len(euroqol_df) > 0:
    euroqol_expandido = euroqol_df.with_columns([
        pl.col('Valor_Original').str.strip_chars().str.split('/').list.first().alias('valor_limpio')
    ]).filter(
        pl.col('valor_limpio').is_not_null() &
        (pl.col('valor_limpio').str.len_chars() == 5)
    ).with_columns([
        pl.col('valor_limpio').str.slice(0, 1).alias('EQ5D_Movilidad'),
        pl.col('valor_limpio').str.slice(1, 1).alias('EQ5D_CuidadoPersonal'),
        pl.col('valor_limpio').str.slice(2, 1).alias('EQ5D_ActividadesHabituales'),
        pl.col('valor_limpio').str.slice(3, 1).alias('EQ5D_DolorMalestar'),
        pl.col('valor_limpio').str.slice(4, 1).alias('EQ5D_AnsiedadDepresion')
    ])

    dimensiones_eq5d = sorted(['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales',
                              'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion'])

    euroqol_expandido = euroqol_expandido.sort(['SIPCOD', 'Fecha'])

    euroqol_largo = euroqol_expandido.unpivot(
        index=['SIPCOD', 'Fecha'] + sorted(cols_demograficas[1:]), 
        on=dimensiones_eq5d,
        variable_name='Variable',
        value_name='Valor'
    )
    
    euroqol_largo = euroqol_largo.sort(['SIPCOD', 'Fecha', 'Variable'])
    
    print(f"  EuroQol expandit: {len(euroqol_largo):,} files")
else:
    euroqol_largo = pl.DataFrame()

# Unir tot
dataframes_finales = []
if len(df_simple_fusionado) > 0:
    dataframes_finales.append(df_simple_fusionado)
    print(f"\nVariables simples: {len(df_simple_fusionado):,} files")

if len(euroqol_largo) > 0:
    dataframes_finales.append(euroqol_largo)
    print(f"EuroQol: {len(euroqol_largo):,} files")

if dataframes_finales:
    columnas_comunes = None
    for df in dataframes_finales:
        if columnas_comunes is None:
            columnas_comunes = sorted(df.columns)
        else:
            columnas_comunes = sorted([col for col in columnas_comunes if col in df.columns])

    print(f"Columnes comunes: {len(columnas_comunes)}")

    # Reordenar columnes
    dataframes_reordenados = [df.select(columnas_comunes) for df in dataframes_finales]
    df_final_mejorado = pl.concat(dataframes_reordenados, how="vertical")

    print(f"\nRESULTAT MILLORAT: {len(df_final_mejorado):,} files totals")
    print(f"Pacients únics: {df_final_mejorado['SIPCOD'].n_unique():,}")

    df_final_mejorado = df_final_mejorado.sort(['SIPCOD', 'Fecha', 'Variable', 'Valor'])

    # Completar dades demogràfiques
    cols_rellenar = sorted([col for col in cols_demograficas[1:] if col in df_final_mejorado.columns])
    if cols_rellenar:
        df_final_mejorado = df_final_mejorado.with_columns([
            pl.col(col).fill_null(strategy="forward").fill_null(strategy="backward").over('SIPCOD')
            for col in cols_rellenar
        ])

    # Eliminar duplicats 
    
    df_final_mejorado = df_final_mejorado.sort(['SIPCOD', 'Fecha', 'Variable', 'Valor'])
    df_final_mejorado = df_final_mejorado.unique(maintain_order=True)

    print(f"RESULTAT FINAL: {df_final_mejorado.shape}")

    # Mostrar distribució per variable
    print(f"\nDISTRIBUCIÓ PER VARIABLE:")
    conteo_final = df_final_mejorado.group_by('Variable').agg(pl.len().alias('count')).sort(['Variable'])  # ORDE ALFABÈTIC
    print(conteo_final)

  
    print(f"Abans: ~33,805 files")
    print(f"Ara: {len(df_final_mejorado):,} files")
    if len(df_final_mejorado) > 0:
        mejora = len(df_final_mejorado) - 33805
        porcentaje = ((len(df_final_mejorado) / 33805) - 1) * 100
        print(f"Increment: +{mejora:,} files ({porcentaje:.1f}% més dades!)")

else:
    print("No s'han pogut processar les dades")
    df_final_mejorado = pl.DataFrame()

# Assignar el resultat final al DataFrame principal
df = df_final_mejorado

# Classificació correcta de variables
variables_euroqol = df.filter(pl.col("Variable").str.starts_with("EQ5D"))["Variable"].unique().sort()
variables_numericas = sorted(["ALBUMINA", "PCR", "CREATININA", "Rockwood"]) 
variables_ordinales = variables_euroqol.to_list()

# Neteja amb càlculs temporals clau
df_clean = df.with_columns([
    # Conversió de dades
    pl.col("Fecha").str.to_datetime(strict=False),
    pl.col("fecha Naci").str.to_datetime(strict=False),
    pl.col("fecha Exitus").str.to_datetime(strict=False),

    # Valors segons tipus de variable
    pl.when(pl.col("Variable").is_in(variables_numericas))
      .then(pl.col("Valor").cast(pl.Float64, strict=False))
      .otherwise(None).alias("valor_numerico"),

    pl.when(pl.col("Variable").is_in(variables_ordinales))
      .then(pl.col("Valor").cast(pl.Int32, strict=False))
      .otherwise(None).alias("valor_ordinal")
]).with_columns([
    # Edat al moment de la mesura
    ((pl.col("Fecha") - pl.col("fecha Naci")).dt.total_days()).alias("edad_dias_medicion"),

    # Estat vital
    pl.col("fecha Exitus").is_not_null().alias("fallecido"),

    # Temps fins a la defunció  o fins a l'última mesura (si viu)
    pl.when(pl.col("fecha Exitus").is_not_null())
      .then((pl.col("fecha Exitus") - pl.col("Fecha")).dt.total_days())
      .otherwise(None).alias("dias_hasta_exitus"),

    # Temps de supervivència des de la mesura fins a l'esdeveniment o censura
    pl.when(pl.col("fecha Exitus").is_not_null())
      .then((pl.col("fecha Exitus") - pl.col("Fecha")).dt.total_days())
      .otherwise((pl.lit("2025-06-27").str.to_date() - pl.col("Fecha").dt.date()).dt.total_days())
      .alias("tiempo_supervivencia")
])

# ORDENAMENT 
df_clean = df_clean.sort(['SIPCOD', 'Fecha', 'Variable', 'Valor'])

print("\nDATASET NET I")
print(f"Forma final: {df_clean.shape}")
print(f"Columnes disponibles: {sorted(df_clean.columns)}") 
print(f"Variables numèriques: {variables_numericas}")
print(f"Variables ordinals EuroQol: {sorted(variables_ordinales)}")  

import hashlib
def crear_hash_verificacion(df):
    """Crea un hash del DataFrame per verificar la reproducibilitat"""
    df_sorted = df.sort(sorted(df.columns))
    data_string = str(df_sorted.to_pandas().to_string())
    hash_resultado = hashlib.md5(data_string.encode()).hexdigest()
    print(f"Hash de verificació: {hash_resultado}")
    return hash_resultado

hash_final = crear_hash_verificacion(df_clean)

In [ ]:
# Llistat de tipus de pacient a conservar
tipos_a_mantener = [
    "Crónico pluripatológico",
"Paliativo no oncológico",
"Paliativo oncológico"
]

# Aplicar el filtre en polars
df_filtrado_polars = df_clean.filter(
    pl.col("TIPO_PACIENTE_FUSIONADO").is_in(tipos_a_mantener)
)


In [ ]:
df_clean=df_filtrado_polars

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
#from jinja2 import Template
import warnings
warnings.filterwarnings('ignore')

# Configuració d'estil
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 11

print("="*80)
print("ANÀLISI DE SUPERVIVÈNCIA")
print("="*80)

# PREPARACIÓ DE DADES
# ===============================================================================

print("\n1. PREPARACIÓ DE DADES")
print("-" * 50)

# Convertir df_clean a format adequat
if hasattr(df_clean, 'to_pandas'):
    df_surv = df_clean.to_pandas()
else:
    df_surv = df_clean.copy()

print(f"Dataset original: {df_surv.shape}")

# Crear dataset per pacient
df_pacients = df_surv.groupby('SIPCOD').agg({
    'edad_dias_medicion': 'first',
    'fallecido': 'first', 
    'tiempo_supervivencia': 'first',
    'sexo': 'first',
    'TIPO_PACIENTE_FUSIONADO': 'first',
    'HOSPITAL_FUSIONADO': 'first',
    'ZONA_FUSIONADA': 'first'
}).reset_index()

# Convertir variables temporals
df_pacients['edad_años'] = df_pacients['edad_dias_medicion'] / 365.25
df_pacients['tiempo_meses'] = df_pacients['tiempo_supervivencia'] / 30.44
df_pacients['evento'] = df_pacients['fallecido'].astype(int)

print(f"Pacients únics: {len(df_pacients)}")

# Afegir variables clíniques -> valor basal (primera mesura disponible)
variables_cliniques = {
    'Rockwood': 'rockwood_basal',
    'ALBUMINA': 'albumina_basal', 
    'PCR': 'pcr_basal',
    'CREATININA': 'creatinina_basal'
}

for var_original, var_nova in variables_cliniques.items():
    # Obtindre primera mesura vàlida per pacient
    primera_mesura = (df_surv[df_surv['Variable'] == var_original]
                       .sort_values(['SIPCOD', 'Fecha'])
                       .groupby('SIPCOD')
                       .first()['valor_numerico'])
    
    df_pacients[var_nova] = df_pacients['SIPCOD'].map(primera_mesura)
    
    if primera_mesura.notna().sum() > 0:
        print(f"{var_original}: {primera_mesura.notna().sum()} pacients amb dades basals")

# Afegir EuroQol
euroqol_vars = [var for var in df_surv['Variable'].unique() if 'EQ5D' in str(var)]
if euroqol_vars:
    
    # Calcular puntuació EuroQol 
    eq5d_data = (df_surv[df_surv['Variable'].isin(euroqol_vars)]
                .sort_values(['SIPCOD', 'Fecha'])
                .groupby('SIPCOD')
                .agg({'valor_ordinal': 'mean'})
                .reset_index())
    
    df_pacients['euroqol_basal'] = df_pacients['SIPCOD'].map(
        eq5d_data.set_index('SIPCOD')['valor_ordinal']
    )
    
    

def calcular_euroqol_espanyol(row):
    """Calcula l'índex EuroQol-5D segons els coeficients espanyols"""
    cols = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales', 
            'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']
    
    # Verificar que totes les columnes existeixen i no són nul·les
    if not all(col in row.index and pd.notna(row[col]) for col in cols):
        return np.nan
    
    try:
        valores = [int(row[col]) for col in cols]
        m, c, a, d, an = valores
        
        # Si tots són 1, índex perfecte
        if all(x == 1 for x in valores):
            return 1.0
        
        # Coeficients  (EQ-5D-3L)
        decrements = {
            'movilidad': [0, 0.0897, 0.1794],     
            'cuidado': [0, 0.1012, 0.2024],
            'actividades': [0, 0.0551, 0.1102], 
            'dolor': [0, 0.0596, 0.1192],
            'ansiedad': [0, 0.0512, 0.1024]
        }
        
        # Calcular decrement total
        decrement_total = (decrements['movilidad'][min(m-1, 2)] + 
                          decrements['cuidado'][min(c-1, 2)] + 
                          decrements['actividades'][min(a-1, 2)] + 
                          decrements['dolor'][min(d-1, 2)] + 
                          decrements['ansiedad'][min(an-1, 2)])
        
        # Penalització addicional si alguna dimensió és nivell 3
        if any(x == 3 for x in valores):
            decrement_total += 0.2119
        
        # Aplicar fórmula: 1 - constant - decrements
        index_eq = 1 - 0.1502 - decrement_total
        
        return max(index_eq, -0.5)  # Mínim teòric
    except:
        return np.nan
    
euroqol_vars = [var for var in df_surv['Variable'].unique() if 'EQ5D' in str(var)]
if euroqol_vars:
    print(f"Variables EuroQol disponibles: {len(euroqol_vars)}")
    
    # Filtrar datos EuroQol
    eq5d_consistent = df_surv[df_surv['Variable'].isin(euroqol_vars)].copy()
    
    if len(eq5d_consistent) > 0:
        # Pivot EQ5D per aplicar la fórmula
        eq_pivot = (eq5d_consistent.pivot_table(
            index=["SIPCOD", "Fecha"],
            columns="Variable",
            values="valor_ordinal",
            aggfunc="first"
        ).reset_index())
        
        # Aplicar fórmula índex
        eq_pivot["eq_index_esp"] = eq_pivot.apply(calcular_euroqol_espanyol, axis=1)
        
        # Mitjana per pacient
        eq_per_pacient = eq_pivot.groupby('SIPCOD')['eq_index_esp'].mean()
        
        # Asignar índex EuroQol a cada paciente
        df_pacients['euroqol_basal'] = df_pacients['SIPCOD'].map(eq_per_pacient)
        
        print(f"EuroQol: {eq_per_pacient.notna().sum()} pacients amb dades d'índex")
    else:
        df_pacients['euroqol_basal'] = np.nan
        print("No hi ha dades EuroQol consistents disponibles")
else:
    df_pacients['euroqol_basal'] = np.nan
    print("No s'han trobat variables EuroQol")

# Netejar dataset final
df_surv_final = df_pacients.dropna(subset=['tiempo_meses', 'evento', 'edad_años', 'ZONA_FUSIONADA']).copy()

# ESTADÍSTIQUES DESCRIPTIVES
# ===============================================================================
# Taula descriptiva
print("CARACTERÍSTIQUES BASALS:")
print(f"  N total: {len(df_surv_final):,}")
print(f"  Esdeveniments (morts): {df_surv_final['evento'].sum():,} ({df_surv_final['evento'].mean()*100:.1f}%)")
print(f"  Temps seguiment mitjà: {df_surv_final['tiempo_meses'].median():.1f} mesos")
print(f"  Rang seguiment: {df_surv_final['tiempo_meses'].min():.1f} - {df_surv_final['tiempo_meses'].max():.1f} mesos")

# Sexe
if 'sexo' in df_surv_final.columns:
    sexo_counts = df_surv_final['sexo'].value_counts()
    print(f"  Sexe: {dict(sexo_counts)}")

# Variables clíniques
print(f"\nVARIABLES CLÍNIQUES BASALS:")
for var_col in ['rockwood_basal', 'albumina_basal', 'pcr_basal', 'creatinina_basal', 'euroqol_basal']:
    if var_col in df_surv_final.columns:
        var_data = df_surv_final[var_col].dropna()
        if len(var_data) > 0:
            print(f"  {var_col.replace('_basal', '').upper()}:")
            print(f"    Disponible: {len(var_data)} ({len(var_data)/len(df_surv_final)*100:.1f}%)")
            print(f"    Mitjana ± DE: {var_data.mean():.2f} ± {var_data.std():.2f}")
            print(f"    Mediana [P25-P75]: {var_data.median():.2f} [{var_data.quantile(0.25):.2f}-{var_data.quantile(0.75):.2f}]")

# ===============================================================================
# 3. SUPERVIVÈNCIA GLOBAL - KAPLAN-MEIER
# ===============================================================================
# Corba Kaplan-Meier global
kmf_global = KaplanMeierFitter()
kmf_global.fit(df_surv_final['tiempo_meses'], df_surv_final['evento'], 
               label=f'Supervivència Global (n={len(df_surv_final)})')

# Mediana supervivència
mediana_global = kmf_global.median_survival_time_
print(f"Mediana supervivència: {mediana_global:.1f} mesos" if not np.isnan(mediana_global) else "Mediana supervivència: No assolida")

# Probabilitats supervivència
print("Probabilitats supervivència:")
for t in [6, 12, 18, 24, 36]:
    if t <= df_surv_final['tiempo_meses'].max():
        try:
            prob = kmf_global.predict(t)
            # Buscar el temps més proper en l'índex
            temps_proper = kmf_global.confidence_interval_survival_function_.index[
                np.abs(kmf_global.confidence_interval_survival_function_.index - t).argmin()
            ]
            ic_lower, ic_upper = kmf_global.confidence_interval_survival_function_.loc[temps_proper]
            print(f"  {t:2d} mesos: {prob:.1%} (IC95%: {ic_lower:.1%}-{ic_upper:.1%})")
        except (KeyError, IndexError):
            prob = kmf_global.predict(t)
            print(f"  {t:2d} mesos: {prob:.1%} (IC95%: no disponible)")

# Gràfic supervivència global
fig, ax = plt.subplots(figsize=(10, 6))
kmf_global.plot_survival_function(ax=ax, color='#2E86AB', linewidth=3)
ax.fill_between(kmf_global.confidence_interval_survival_function_.index,
                kmf_global.confidence_interval_survival_function_.iloc[:, 0],
                kmf_global.confidence_interval_survival_function_.iloc[:, 1],
                alpha=0.2, color='#2E86AB')

ax.set_title('Corba de Supervivència Global', fontsize=16, fontweight='bold')
ax.set_xlabel('Temps (mesos)', fontsize=12)
ax.set_ylabel('Probabilitat de Supervivència', fontsize=12)
ax.set_xticks([0, 6, 12, 18, 24] + list(range(30, 145, 10)))
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)

# Afegir 
ax.text(0.02, 0.98, f'N = {len(df_surv_final):,}\nEsdeveniments = {df_surv_final["evento"].sum():,} ({df_surv_final["evento"].mean()*100:.1f}%)', 
        transform=ax.transAxes, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# DEFINICIÓ DE FUNCIONS AUXILIARS
# ===============================================================================

def analitzar_supervivencia_grups(df, variable, titol, categories=None, colors=None):
    """Funció per a anàlisi supervivència per grups amb corbes KM"""
    
    if variable not in df.columns:
        print(f"Variable {variable} no disponible")
        return
    
    # Filtrar dades vàlides
    df_grup = df.dropna(subset=[variable, 'tiempo_meses', 'evento']).copy()
    
    if categories is not None:
        # Convertir a category si no ho és
        if not hasattr(df_grup[variable], 'cat'):
            df_grup[variable] = df_grup[variable].astype('category')
        
        # Reordenar categories
        df_grup[variable] = df_grup[variable].cat.reorder_categories(categories)
        grups = categories
    else:
        # Si no hi ha categories especificades, usar l'ordre original
        if hasattr(df_grup[variable], 'cat'):
            grups = df_grup[variable].cat.categories
        else:
            grups = sorted(df_grup[variable].unique())
    
    if len(grups) < 2:
        print(f"Grups insuficients per a {variable}")
        return
    
    print(f"\n{titol}:")
    
    # Colors
    if colors is None:
        colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#6A994E']
    
    # Crear gràfics
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Corbes Kaplan-Meier
    kmf = KaplanMeierFitter()
    resultats_grups = []
    
    for i, grup in enumerate(grups):
        if i >= len(colors):
            break
            
        data_grup = df_grup[df_grup[variable] == grup]
        n_grup = len(data_grup)
        events_grup = data_grup['evento'].sum()
        
        if n_grup >= 5:  # Mínim 5 pacients
            kmf.fit(data_grup['tiempo_meses'], data_grup['evento'],
                   label=f'{grup} (n={n_grup})')
            kmf.plot_survival_function(ax=axes[0], color=colors[i], linewidth=2.5)
            
            # Estadístiques
            mediana = kmf.median_survival_time_
            try:
                prob_12m = kmf.predict(12) if 12 <= data_grup['tiempo_meses'].max() else None
            except:
                prob_12m = None
            try:
                prob_24m = kmf.predict(24) if 24 <= data_grup['tiempo_meses'].max() else None
            except:
                prob_24m = None
            
            resultats_grups.append({
                'grup': grup,
                'n': n_grup,
                'events': events_grup,
                'tasa_events': events_grup/n_grup*100,
                'mediana': mediana if not np.isnan(mediana) else None,
                'prob_12m': prob_12m,
                'prob_24m': prob_24m
            })
    
    # Configurar gràfic corbes
    axes[0].set_title(titol, fontsize=14, fontweight='bold', pad=20)
    axes[0].set_xlabel('Temps (mesos)')
    axes[0].set_ylabel('Probabilitat Supervivència')
    axes[0].set_xticks([0, 6, 12, 18, 24] + list(range(30, 145, 10)))
    axes[0].legend(loc='upper right')
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim(0, 1.05)
    
    # Gràfic taxes esdeveniments
    if resultats_grups:
        df_res = pd.DataFrame(resultats_grups)
        
        bars = axes[1].bar(range(len(df_res)), df_res['tasa_events'],
                          color=colors[:len(df_res)], alpha=0.8, edgecolor='black')
        
        axes[1].set_xticks(range(len(df_res)))
        axes[1].set_xticklabels(df_res['grup'], rotation=45, ha='right')
        axes[1].set_ylabel('Taxa Esdeveniments (%)')
        axes[1].set_title('Taxa Esdeveniments per Grup', fontweight='bold', pad=20)
        axes[1].grid(True, alpha=0.3, axis='y')
        
        # Afegir valors en barres
        for bar, tasa, n in zip(bars, df_res['tasa_events'], df_res['n']):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(df_res['tasa_events'])*0.02,
                        f'{tasa:.1f}%\n(n={n})', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Test log-rank
    if len(grups) >= 2 and len(df_grup) > 20:
        try:
            # Test bivariant entre dos grups principals
            grup1, grup2 = grups[0], grups[1]
            
            data1 = df_grup[df_grup[variable] == grup1]
            data2 = df_grup[df_grup[variable] == grup2]
            
            if len(data1) >= 5 and len(data2) >= 5:
                result = logrank_test(
                    data1['tiempo_meses'], data2['tiempo_meses'],
                    data1['evento'], data2['evento']
                )
                
                print(f"  Test Log-rank ({grup1} vs {grup2}): p = {result.p_value:.4f}")
                if result.p_value < 0.05:
                    print(f"   Diferències significatives entre grups")
                else:
                    print(f"   No hi ha diferències significatives")
        except Exception as e:
            print(f"  Error en test log-rank: {str(e)[:50]}")
    
    # Mostrar estadístiques
    if resultats_grups:
        print(f"  Estadístiques per grup:")
        for res in resultats_grups:
            mediana_str = f"{res['mediana']:.1f} mesos" if res['mediana'] else "No assolida"
            print(f"    {res['grup']}: {res['events']}/{res['n']} events ({res['tasa_events']:.1f}%), mediana: {mediana_str}")

# ANÀLISI PER GRUPS (CORBES SUPERVIVÈNCIA)
# ===============================================================================
# Crear grups per a anàlisi 

# Grups d'edat
df_surv_final['grup_edat'] = pd.cut(df_surv_final['edad_años'], 
                                    bins=[0, 70, 80, 90, 150],
                                    labels=['<70 anys', '70-80 anys', '80-90 anys', '>90 anys'], ordered=True)

analitzar_supervivencia_grups(df_surv_final, 'grup_edat', 'Supervivència per Edat')

# Grups sexe
if 'sexo' in df_surv_final.columns:
    df_surv_final['grup_sexe'] = df_surv_final['sexo'].map({'1': 'Homes', '2': 'Dones'})
    analitzar_supervivencia_grups(df_surv_final, 'grup_sexe', 'Supervivència per Sexe', 
                                 colors=['#2E86AB', '#F18F01'])

# Grups fragilitat
if 'rockwood_basal' in df_surv_final.columns and df_surv_final['rockwood_basal'].notna().sum() > 30:
    rockwood_stats = df_surv_final['rockwood_basal'].describe()
    print(f"\nEstadístiques Rockwood: min={rockwood_stats['min']:.2f}, max={rockwood_stats['max']:.2f}")
    
    q25, q50, q75 = df_surv_final['rockwood_basal'].quantile([0.25, 0.5, 0.75])
    df_surv_final['grup_rockwood'] = pd.cut(df_surv_final['rockwood_basal'],
                                           bins=[0, q25, q75, 1.0],
                                           labels=[f'Fragilitat dèbil (≤{q25:.2f})', 
                                                  f'Fragilitat moderada ({q25:.2f}-{q75:.2f})', 
                                                  f'Fragilitat severa (>{q75:.2f})'],ordered=True)
    
    analitzar_supervivencia_grups(df_surv_final, 'grup_rockwood', 'Supervivència per Fragilitat (Rockwood)')

# Grups albúmina
if 'albumina_basal' in df_surv_final.columns and df_surv_final['albumina_basal'].notna().sum() > 30:
    df_surv_final['grup_albumina'] = pd.cut(df_surv_final['albumina_basal'],
                                           bins=[0, 3.0, 3.5, 10],
                                           labels=['Desnutrició (<3.0)', 'Límit (3.0-3.5)', 'Normal (>3.5)'], ordered=True)
    
    analitzar_supervivencia_grups(df_surv_final, 'grup_albumina', 'Supervivència per Albúmina')

# Grups PCR
if 'pcr_basal' in df_surv_final.columns and df_surv_final['pcr_basal'].notna().sum() > 30:
    df_surv_final['grup_pcr'] = pd.cut(df_surv_final['pcr_basal'],
                                       bins=[0, 10, 50, 1000],
                                       labels=['Normal (<10)', 'Elevada (10-50)', 'Molt elevada (>50)'], ordered=True)
    
    analitzar_supervivencia_grups(df_surv_final, 'grup_pcr', 'Supervivència per PCR (Inflamació)')

# Grups creatinina
if 'creatinina_basal' in df_surv_final.columns and df_surv_final['creatinina_basal'].notna().sum() > 30:
    df_surv_final['grup_creatinina'] = pd.cut(df_surv_final['creatinina_basal'],
                                              bins=[0, 1.2, 2.0, 20],
                                              labels=['Normal (<1.2)', 'Elevada (1.2-2.0)', 'Molt elevada (>2.0)'], ordered=True)
    
    analitzar_supervivencia_grups(df_surv_final, 'grup_creatinina', 'Supervivència per Creatinina')

# Grups EuroQol (qualitat de vida)
if 'euroqol_basal' in df_surv_final.columns and df_surv_final['euroqol_basal'].notna().sum() > 30:
    eq_q25, eq_q75 = df_surv_final['euroqol_basal'].quantile([0.25, 0.75])
    df_surv_final['grup_euroqol'] = pd.cut(df_surv_final['euroqol_basal'],
                                           bins=[0, eq_q25, eq_q75, 5],
                                           labels=[f'Baixa QV(<{eq_q25:.2f})', 
                                                  f'Moderada QV ({eq_q25:.1f}-{eq_q75:.2f})', 
                                                  f'Alta QV(>{eq_q75:.2f})'], ordered=True)
    
    analitzar_supervivencia_grups(df_surv_final, 'grup_euroqol', 'Supervivència per Qualitat de Vida (EuroQol)')

# Anàlisi per tipus de pacient
if 'TIPO_PACIENTE_FUSIONADO' in df_surv_final.columns:
    tipus_freq = df_surv_final['TIPO_PACIENTE_FUSIONADO'].value_counts()
    tipus_principals = tipus_freq[tipus_freq >= 50].index[:6]
    
    if len(tipus_principals) >= 2:
        df_tipus = df_surv_final[df_surv_final['TIPO_PACIENTE_FUSIONADO'].isin(tipus_principals)].copy()
        print(f"\nTipus de pacient analitzats: {len(tipus_principals)}")
        for tipus in tipus_principals:
            n_tipus = (df_tipus['TIPO_PACIENTE_FUSIONADO'] == tipus).sum()
            print(f"  {tipus}: {n_tipus} pacients")
        
        analitzar_supervivencia_grups(df_tipus, 'TIPO_PACIENTE_FUSIONADO',
                                     'Supervivència per Tipus de Patologia')

# Anàlisi per hospital
if 'HOSPITAL_FUSIONADO' in df_surv_final.columns:
    hospitals_freq = df_surv_final['HOSPITAL_FUSIONADO'].value_counts()
    hospitals_principals = hospitals_freq[hospitals_freq >= 100].index[:5]
    
    if len(hospitals_principals) >= 2:
        df_hospitals = df_surv_final[df_surv_final['HOSPITAL_FUSIONADO'].isin(hospitals_principals)].copy()
        print(f"\nHospitals analitzats: {len(hospitals_principals)}")
        
        analitzar_supervivencia_grups(df_hospitals, 'HOSPITAL_FUSIONADO',
                                     'Supervivència per Hospital')

# ===============================================================================
# 5. ESTRATIFICACIÓ DE RISC
# ===============================================================================

# Score clínic  
df_surv_final['score_clinico'] = 0
factors_inclosos = []

# Edat >85 anys
edad_p75 = df_surv_final['edad_años'].quantile(0.75)
df_surv_final['score_clinico'] += (df_surv_final['edad_años'] > 90).astype(int)
factors_inclosos.append(f"Edat > 90 anys")

# Sexe masculí 
if 'sexo_masculino' in df_surv_final.columns:
    df_surv_final['score_clinico'] += df_surv_final['sexo_masculino'].fillna(0).astype(int)
    factors_inclosos.append("Sexe masculí")
elif 'sexo' in df_surv_final.columns:
    df_surv_final['score_clinico'] += (df_surv_final['sexo'] == '1').fillna(False).astype(int)
    factors_inclosos.append("Sexe masculí")

# Tipus de Pacient
if 'TIPO_PACIENTE_FUSIONADO' in df_surv_final.columns:
    df_surv_final['score_clinico'] += (df_surv_final['TIPO_PACIENTE_FUSIONADO'] == 'Paliativo oncológico').fillna(False).astype(int)
    factors_inclosos.append("Pacient pal·liatiu oncològic")

# Fragilitat alta 
if 'rockwood_basal' in df_surv_final.columns:
    rockwood_p75 = df_surv_final['rockwood_basal'].quantile(0.75)
    df_surv_final['score_clinico'] += (df_surv_final['rockwood_basal'] > 0.5).fillna(False).astype(int)
    factors_inclosos.append(f"Rockwood > 0.5")

# Albúmina baixa
if 'albumina_basal' in df_surv_final.columns:
    df_surv_final['score_clinico'] += (df_surv_final['albumina_basal'] < 3.0).fillna(False).astype(int)
    factors_inclosos.append("Albúmina < 3.0")

# PCR alta
if 'pcr_basal' in df_surv_final.columns:
    pcr_p75 = df_surv_final['pcr_basal'].quantile(0.75)
    df_surv_final['score_clinico'] += (df_surv_final['pcr_basal'] > 50.0).fillna(False).astype(int)
    factors_inclosos.append(f"PCR > 50.0")

# Creatinina alta
if 'creatinina_basal' in df_surv_final.columns:
    df_surv_final['score_clinico'] += (df_surv_final['creatinina_basal'] > 2.0).fillna(False).astype(int)
    factors_inclosos.append("Creatinina > 2.0")

if 'euroqol_basal' in df_surv_final.columns:
    df_surv_final['score_clinico'] += (df_surv_final['euroqol_basal'] < 0.18).fillna(False).astype(int)
    factors_inclosos.append("EuroQol < 0.18")

if len(factors_inclosos) >= 3:
    print(f"  Factors inclosos: {', '.join(factors_inclosos)}")
    
    # Distribució del score
    score_dist = df_surv_final['score_clinico'].value_counts()
    print(f"  Distribució score:")
    for score, count in score_dist.items():
        print(f"    {score} factors: {count} pacients ({count/len(df_surv_final)*100:.1f}%)")
    
    # Crear grups de risc clínic
    conditions = [
        (df_surv_final['score_clinico'] <= 1),
        (df_surv_final['score_clinico'] == 2),
        (df_surv_final['score_clinico'] >= 3)
    ]
    choices = ['Baix (0-1 factors)', 'Intermedi (2 factors)', 'Sever (≥3 factors)']
    
    df_surv_final['riesgo_clinico'] = np.select(conditions, choices, default='Sense classificar')
   
    # Anàlisi supervivència per score clínic
    analitzar_supervivencia_grups(df_surv_final, 'riesgo_clinico',
                                 'Supervivència per Score Clínic de Risc',
                                 colors=['#2ca02c', '#ff7f0e','#d62728'])




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test
from scipy import stats
from scipy.stats import spearmanr, pearsonr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuració d'estil
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 11


# APLICAR FILTRE DE TIPUS DE PACIENT
# ===============================================================================


# Llistat de tipus de pacient a conservar
tipus_a_mantenir = [
    "Crónico pluripatológico",
    "Paliativo no oncológico", 
    "Paliativo oncológico"
]

# Aplicar el filtre
if hasattr(df_clean, 'to_pandas'):
    df_work = df_clean.to_pandas()
else:
    df_work = df_clean.copy()

print(f"Dataset original: {len(df_work)} registres")

# Filtrar per tipus de pacient
df_filtrat = df_work[df_work['TIPO_PACIENTE_FUSIONADO'].isin(tipus_a_mantenir)].copy()

print(f"Dataset filtrat: {len(df_filtrat)} registres")

print(f"\nDistribució per tipus de pacient:")
tipus_counts = df_filtrat['TIPO_PACIENTE_FUSIONADO'].value_counts()
for tipus, count in tipus_counts.items():
    print(f"  {tipus}: {count:,} ({count/len(df_filtrat)*100:.1f}%)")

#PREPARACIÓ DE DATASET PER PACIENT
# ===============================================================================

# Crear dataset per pacient
df_pacients = df_filtrat.groupby('SIPCOD').agg({
    'edad_dias_medicion': 'first',
    'fallecido': 'first',
    'tiempo_supervivencia': 'first', 
    'sexo': 'first',
    'TIPO_PACIENTE_FUSIONADO': 'first',
    'HOSPITAL_FUSIONADO': 'first',
    'ZONA_FUSIONADA': 'first'
}).reset_index()

# Conversions temporals
df_pacients['edat_anys'] = df_pacients['edad_dias_medicion'] / 365.25
df_pacients['temps_mesos'] = df_pacients['tiempo_supervivencia'] / 30.44
df_pacients['event'] = df_pacients['fallecido'].astype(int)

print(f"Pacients únics: {len(df_pacients)}")

# Mostrar informació de la variable edat_anys
if 'edat_anys' in df_pacients.columns:
    edat_valida = df_pacients['edat_anys'].dropna()
    if len(edat_valida) > 0:
        print(f"\nEDAT_ANYS:")
        print(f"  N vàlids: {len(edat_valida)}")
        print(f"  Rang: {edat_valida.min():.2f} - {edat_valida.max():.2f} anys")
        print(f"  Mitjana ± DE: {edat_valida.mean():.2f} ± {edat_valida.std():.2f}")
        print(f"  Mediana [P25-P75]: {edat_valida.median():.2f} [{edat_valida.quantile(0.25):.2f}-{edat_valida.quantile(0.75):.2f}]")

# ANÀLISI I CORRECCIÓ D'ESCALES CLÍNIQUES
# ===============================================================================
# Obtindre valors basals (primera mesura per pacient)
variables_cliniques = {
    'Rockwood': 'rockwood_basal',
    'ALBUMINA': 'albumina_basal',
    'PCR': 'pcr_basal', 
    'CREATININA': 'creatinina_basal'
}

for var_original, var_nova in variables_cliniques.items():
    primera_mesura = (df_filtrat[df_filtrat['Variable'] == var_original]
                      .sort_values(['SIPCOD', 'Fecha'])
                      .groupby('SIPCOD')
                      .first()['valor_numerico'])
    
    df_pacients[var_nova] = df_pacients['SIPCOD'].map(primera_mesura)
    
    if primera_mesura.notna().sum() > 0:
        valors_valids = primera_mesura.dropna()
        print(f"\n{var_original}:")
        print(f"  N vàlids: {len(valors_valids)}")
        print(f"  Rang: {valors_valids.min():.3f} - {valors_valids.max():.3f}")
        print(f"  Mitjana ± DE: {valors_valids.mean():.3f} ± {valors_valids.std():.3f}")
        print(f"  Mediana [P25-P75]: {valors_valids.median():.3f} [{valors_valids.quantile(0.25):.3f}-{valors_valids.quantile(0.75):.3f}]")

# EuroQol 
def calcular_euroqol_espanyol(row):
    """Calcula l'índex EuroQol-5D segons els coeficients espanyols"""
    cols = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales', 
            'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']
    
    # Verificar que totes les columnes existeixen i no són nul·les
    if not all(col in row.index and pd.notna(row[col]) for col in cols):
        return np.nan
    
    try:
        valors = [int(row[col]) for col in cols]
        m, c, a, d, an = valors
        
        # Si tots són 1, índex perfecte
        if all(x == 1 for x in valors):
            return 1.0
        
        # Coeficients  (EQ-5D-3L)
        decrements = {
            'movilitat': [0, 0.0897, 0.1794],      
            'cuidat': [0, 0.1012, 0.2024],
            'activitats': [0, 0.0551, 0.1102], 
            'dolor': [0, 0.0596, 0.1192],
            'ansietat': [0, 0.0512, 0.1024]
        }
        
        # Calcular decrement total
        decrement_total = (decrements['movilitat'][min(m-1, 2)] + 
                          decrements['cuidat'][min(c-1, 2)] + 
                          decrements['activitats'][min(a-1, 2)] + 
                          decrements['dolor'][min(d-1, 2)] + 
                          decrements['ansietat'][min(an-1, 2)])
        
        # Penalització addicional si alguna dimensió és nivell 3
        if any(x == 3 for x in valors):
            decrement_total += 0.2119
        
        # Aplicar fórmula: 1 - constant - decrements
        index_eq = 1 - 0.1502 - decrement_total
        
        # L'índex negatiu (estats pitjors que la mort)
        return max(index_eq, -0.5)  # Mínim teòric
    except:
        return np.nan

euroqol_vars = [var for var in df_filtrat['Variable'].unique() if 'EQ5D' in str(var)]

if euroqol_vars:
    eq5d_data = (df_filtrat[df_filtrat['Variable'].isin(euroqol_vars)]
                .sort_values(['SIPCOD', 'Fecha'])
                .groupby('SIPCOD')
                .agg({'valor_ordinal': 'mean'})
                .reset_index())
    
if euroqol_vars:
    # Crear un pivot per tindre les dimensions com a columnes
    eq5d_pivot = (df_filtrat[df_filtrat['Variable'].isin(euroqol_vars)]
                  .pivot_table(index='SIPCOD', 
                             columns='Variable', 
                             values='valor_ordinal',
                             aggfunc='first')  # o 'mean' si hi ha múltiples valors
                  .reset_index())
    
    # Aplicar la funció a cada fila
    eq5d_pivot['euroqol_basal'] = eq5d_pivot.apply(calcular_euroqol_espanyol, axis=1)
    
    # Mapejar al df principal
    df_pacients['euroqol_basal'] = df_pacients['SIPCOD'].map(
        eq5d_pivot.set_index('SIPCOD')['euroqol_basal']
    )

    euroqol_valids = df_pacients['euroqol_basal'].dropna()
    if len(euroqol_valids) > 0:
        print(f"\nEUROQOL:")
        print(f"  N vàlids: {len(euroqol_valids)}")
        print(f"  Rang: {euroqol_valids.min():.3f} - {euroqol_valids.max():.3f}")
        print(f"  Mitjana ± DE: {euroqol_valids.mean():.3f} ± {euroqol_valids.std():.3f}")

# DETECCIÓ DE PROBLEMES D'ESCALA I CORRECCIÓ
# ===============================================================================

df_corregit = df_pacients.copy()

# EDAT: Verificar que està en anys (rang esperat 0-120)
if 'edat_anys' in df_corregit.columns:
    edat_orig = df_corregit['edat_anys'].dropna()
    if len(edat_orig) > 0:
        print(f" EDAT: Rang {edat_orig.min():.2f} - {edat_orig.max():.2f} anys")
        if edat_orig.min() >= 0 and edat_orig.max() <= 120:
            print("    Valors raonables per edat en anys")
        else:
            print("     Valors fora del rang fisiològic esperat (0-120 anys)")
        
        print(f"   Mitjana: {edat_orig.mean():.1f} anys")
        print(f"   Mediana: {edat_orig.median():.1f} anys")

# ROCKWOOD:
if 'rockwood_basal' in df_corregit.columns:
    rockwood_orig = df_corregit['rockwood_basal'].dropna()
    if len(rockwood_orig) > 0 and rockwood_orig.max() <= 1.0:
        # Mantenir l'escala
        df_corregit['rockwood_basal'] = df_corregit['rockwood_basal'] 
        
        rockwood_corr = df_corregit['rockwood_basal'].dropna()
        print(f"   Escala original: {rockwood_orig.min():.3f} - {rockwood_orig.max():.3f}")
        print(f"   Escala mantinguda: {rockwood_corr.min():.3f} - {rockwood_corr.max():.3f}")
        print(f"   Interpretació: 0=Molt Fit, 1=Terminalment malalt")
    else:
        df_corregit['rockwood_basal'] = df_corregit['rockwood_basal']

# ALBÚMINA: Verificar escala g/dL (normal 3.5-5.0)
if 'albumina_basal' in df_corregit.columns:
    albumina_orig = df_corregit['albumina_basal'].dropna()
    if len(albumina_orig) > 0:
        print(f" ALBÚMINA: Rang {albumina_orig.min():.2f} - {albumina_orig.max():.2f} g/dL")
        df_corregit['albumina_basal'] = df_corregit['albumina_basal']
        if albumina_orig.min() < 1.0 or albumina_orig.max() > 6.0:
            print("     Valors fora del rang fisiològic esperat")

# PCR: Verificar escala mg/L
if 'pcr_basal' in df_corregit.columns:
    pcr_orig = df_corregit['pcr_basal'].dropna()
    if len(pcr_orig) > 0:
        print(f" PCR: Rang {pcr_orig.min():.2f} - {pcr_orig.max():.2f} mg/L")
        df_corregit['pcr_basal'] = df_corregit['pcr_basal']

# CREATININA: Verificar escala mg/dL (normal 0.6-1.2)
if 'creatinina_basal' in df_corregit.columns:
    creatinina_orig = df_corregit['creatinina_basal'].dropna()
    if len(creatinina_orig) > 0:
        print(f" CREATININA: Rang {creatinina_orig.min():.2f} - {creatinina_orig.max():.2f} mg/dL")
        df_corregit['creatinina_basal'] = df_corregit['creatinina_basal']

# EUROQOL: Verificar escala (hauria d'estar 0-1 o negatiu)
if 'euroqol_basal' in df_corregit.columns:
    euroqol_orig = df_corregit['euroqol_basal'].dropna()
    if len(euroqol_orig) > 0:
        print(f" EUROQOL: Rang {euroqol_orig.min():.3f} - {euroqol_orig.max():.3f}")
        df_corregit['euroqol_basal'] = df_corregit['euroqol_basal']

# ANÀLISI DE COL·LINEALITAT
# ===============================================================================

# Seleccionar variables clíniques corregides per anàlisi
variables_numeriques = ['edat_anys']
if 'sexo' in df_corregit.columns:
    df_corregit['sexe_masculi'] = (df_corregit['sexo'] == '1').astype(int)
    variables_numeriques.append('sexe_masculi')

# Afegir variables clíniques corregides disponibles
variables_cliniques_corregides = []
for var in ['rockwood_basal', 'albumina_basal', 
           'pcr_basal', 'creatinina_basal', 'euroqol_basal']:
    if var in df_corregit.columns and df_corregit[var].notna().sum() > 50:
        variables_cliniques_corregides.append(var)

totes_variables = variables_numeriques + variables_cliniques_corregides

print(f"Variables per anàlisi: {len(totes_variables)}")
for var in totes_variables:
    n_valids = df_corregit[var].notna().sum()
    print(f"  {var}: {n_valids} valors vàlids")

# Dataset complet per anàlisi
df_complet = df_corregit[totes_variables + ['temps_mesos', 'event']].dropna()
print(f"\nDataset complet: {len(df_complet)} pacients")

# VARIABLES PER REGRESSIÓ
# ===============================================================================

df_sense_estandarditzar = df_complet.copy()

print(f"Variables:")
print(f"  - Creatinina: unitats mg/dL")
print(f"  - PCR: unitats mg/L") 
print(f"  - Albúmina: unitats g/dL")
print(f"  - Rockwood: escala 0-1")
print(f"  - EuroQol: escala -0.5 a 1.0")
print(f"  - Edat: anys")
print(f"  - Sexe: variable dicotòmica")

# Crear llista final de variables per Cox
variables_cox_originals = (['edat_anys'] + 
                          [v for v in variables_numeriques if 'sexe' in v] + 
                          variables_cliniques_corregides)

print(f"\nVariables per model Cox: {len(variables_cox_originals)}")
for var in variables_cox_originals:
    print(f"  - {var}")

# MODELS DE COX
# ===============================================================================

variables_cox_finals = [v for v in variables_cox_originals if v != 'sexe_masculi' or df_complet[v].sum() > 10]

try:
    cph_original = CoxPHFitter()
    df_cox_orig = df_sense_estandarditzar[variables_cox_finals + ['temps_mesos', 'event']].dropna()
    
    print(f"  N pacients: {len(df_cox_orig)}")
    
    cph_original.fit(df_cox_orig, duration_col='temps_mesos', event_col='event')
    
    print("  RESULTATS:")
    print("  " + "="*50)
    
    summary_orig = cph_original.summary
    for var, row in summary_orig.iterrows():
        hr = np.exp(row['coef'])
        ic_low = np.exp(row['coef lower 95%'])
        ic_high = np.exp(row['coef upper 95%'])
        
        significatiu = "***" if row['p'] < 0.001 else "**" if row['p'] < 0.01 else "*" if row['p'] < 0.05 else ""
        
        print(f"    {var}: HR={hr:.3f} (IC95%: {ic_low:.3f}-{ic_high:.3f}) p={row['p']:.4f} {significatiu}")
        
        # Interpretacions específiques per variable
        if 'rockwood' in var.lower():
            print(f"       Interpretació: per cada 1 punt d'increment en fragilitat")
        elif 'creatinina' in var.lower():
            print(f"       Interpretació: per cada 1 mg/dL d'increment")
        elif 'pcr' in var.lower():
            print(f"       Interpretació: per cada 1 mg/L d'increment")
        elif 'albumina' in var.lower():
            print(f"       Interpretació: per cada 1 g/dL d'increment")
        elif 'euroqol' in var.lower():
            print(f"       Interpretació: per cada 0.1 punt d'increment en qualitat de vida")
        elif 'edat' in var.lower():
            print(f"       Interpretació: per cada any d'increment")
    
    print(f"  C-index: {cph_original.concordance_index_:.3f}")
    
    # Forest plot del model sense estandarditzar
    fig, ax = plt.subplots(figsize=(12, 8))
    cph_original.plot(ax=ax, hazard_ratios=True)
    ax.set_title('Gràfic Forest - Model Sense Estandarditzar\nHazard Ratios (IC 95%)', 
                 fontsize=16, fontweight='bold')
    ax.axvline(x=1, color='red', linestyle='--', alpha=0.7, linewidth=2)
    ax.set_xlabel('Hazard Ratio')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"  Error en model original: {str(e)}")
    cph_original = None

# ANÀLISI DE MULTICOL·LINEALITAT 
# ===============================================================================

print(f"\n8. ELIMINACIÓ DE MULTICOL·LINEALITAT SEVERA")
print("-" * 50)

if 'cph_original' in locals() and cph_original is not None:
    
    print(f"\n CRITERIS D'INTERPRETACIÓ DEL VIF:")
    print(f"   VIF < 2.5:   Cap problema de multicol·linealitat")
    print(f"   VIF 2.5-5:   Multicol·linealitat lleugera")
    print(f"   VIF 5-10:    Multicol·linealitat moderada")
    print(f"   VIF > 10:    Multicol·linealitat severa")
    
    # Identificar només variables amb VIF > 10 (multicol·linealitat severa)
    variables_vif_sever = []
    if 'vif_data' in locals():
        vif_df = pd.DataFrame(vif_data)
        variables_vif_sever = vif_df[vif_df['VIF'] > 10]['Variable'].tolist()
        
        print(f"\n ANÀLISI VIF:")
        for _, row in vif_df.iterrows():
            var = row['Variable']
            vif_val = row['VIF']
            
            if vif_val > 10:
                status = " SEVERA - ELIMINAR"
                accion = " Eliminar del model"
            elif vif_val > 5:
                status = " MODERADA - MANTENIR"
                accion = " Mantenir amb vigilància"
            elif vif_val > 2.5:
                status = " LLEUGERA - MANTENIR"
                accion = " Mantenir"
            else:
                status = " OK - MANTENIR"
                accion = " Mantenir"
                
            print(f"   {var}: VIF = {vif_val:.2f} - {status}")
            print(f"      Acció: {accion}")
        
        if variables_vif_sever:
            print(f"\n  Variables amb multicol·linealitat severa (VIF > 10):")
            for var in variables_vif_sever:
                vif_val = vif_df[vif_df['Variable'] == var]['VIF'].iloc[0]
                r_squared = 1 - (1/vif_val)
                print(f"   - {var}: VIF = {vif_val:.2f} (R² = {r_squared:.3f})")
                print(f"     Interpretació: Aquesta variable es pot explicar en un {r_squared*100:.1f}% per les altres variables")
            
            # Model sense variables amb VIF sever
            variables_sense_vif_sever = [v for v in variables_cox_finals 
                                        if v not in variables_vif_sever]
            
            print(f"\n MODEL SENSE MULTICOL·LINEALITAT SEVERA:")
            print(f"Variables eliminades: {variables_vif_sever}")
            print(f"Variables mantingudes: {variables_sense_vif_sever}")
            
            if len(variables_sense_vif_sever) >= 2:
                try:
                    cph_sense_multicolineal = CoxPHFitter()
                    df_sense_multicolineal = df_sense_estandarditzar[variables_sense_vif_sever + ['temps_mesos', 'event']].dropna()
                    
                    cph_sense_multicolineal.fit(df_sense_multicolineal, duration_col='temps_mesos', event_col='event')
                    
                    print(f"\n   RESULTATS MODEL:")
                    print(f"   N pacients: {len(df_sense_multicolineal)}")
                    print(f"   C-index: {cph_sense_multicolineal.concordance_index_:.3f}")
                    
                    summary_corregit = cph_sense_multicolineal.summary
                    for var, row in summary_corregit.iterrows():
                        hr = np.exp(row['coef'])
                        significatiu = "***" if row['p'] < 0.001 else "**" if row['p'] < 0.01 else "*" if row['p'] < 0.05 else ""
                        print(f"     {var}: HR={hr:.3f} p={row['p']:.4f} {significatiu}")
                        
                        
                        
                        
                except Exception as e:
                    print(f"   Error: {str(e)}")
            else:
                print("     Insuficients variables per crear model")
        else:
            print(f"\n CAP VARIABLE AMB MULTICOL·LINEALITAT SEVERA")
            print(f"   Totes les variables tenen VIF ≤ 10")
            print(f"   El model original és acceptable des del punt de vista de multicol·linealitat")




In [ ]:
import pandas as pd
import numpy as np
import polars as pl
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PREPARACIÓ DE DADES PER A MODELATGE PREDICTIU
# ============================================================================

# Verificar que tenim el DataFrame net
if 'df_clean' in locals():
    df = df_clean
else:
    print(" No es troba df_clean. Executa primer el codi de neteja.")
    exit()

# Convertir a pandas per facilitar el modelatge
df_pandas = df.to_pandas()

# Anàlisi de mortalitat
print(f" Total pacients: {df_pandas['SIPCOD'].nunique():,}")
print(f" Taxa de mortalitat: {df_pandas.groupby('SIPCOD')['fallecido'].first().mean():.1%}")

def crear_features_pacient_sense_imputacio(df):
    """Crea features agregades per pacient per a modelatge predictiu sense imputació"""
    
    # Agrupar per pacient
    pacients_df = []
    
    for sipcod in df['SIPCOD'].unique():
        dades_pacient = df[df['SIPCOD'] == sipcod].copy()
        
        # Variables demogràfiques
        demo_cols = ['sexo', 'fecha Naci', 'fecha Exitus', 'fallecido', 
                     'TIPO_PACIENTE_FUSIONADO', 'HOSPITAL_FUSIONADO', 'ZONA_FUSIONADA']
        features_pacient = dades_pacient[demo_cols].iloc[0].to_dict()
        features_pacient['SIPCOD'] = sipcod
        
        # Edat 
        if not dades_pacient['edad_dias_medicion'].isna().all():
            edat_primera = dades_pacient['edad_dias_medicion'].dropna().iloc[0]
            features_pacient['edat_anys'] = edat_primera / 365.25
        else:
            features_pacient['edat_anys'] = np.nan
        
        # Features de variables numèriques 
        for variable in ['ALBUMINA', 'PCR', 'CREATININA', 'Rockwood']:
            dades_var = dades_pacient[dades_pacient['Variable'] == variable]['valor_numerico'].dropna()
            
            if len(dades_var) > 0:
                features_pacient[f'{variable}_primera'] = dades_var.iloc[0]
                features_pacient[f'{variable}_mitjana'] = dades_var.mean()
                features_pacient[f'{variable}_max'] = dades_var.max()
                features_pacient[f'{variable}_min'] = dades_var.min()
                features_pacient[f'{variable}_n_mesures'] = len(dades_var)
                
                if len(dades_var) > 1:
                    features_pacient[f'{variable}_tendencia'] = (dades_var.iloc[-1] - dades_var.iloc[0]) / len(dades_var)
                else:
                    features_pacient[f'{variable}_tendencia'] = 0
                    
                # Flag de disponibilitat
                features_pacient[f'{variable}_disponible'] = 1
            else:
                # No crear features per a variables sense dades
                features_pacient[f'{variable}_disponible'] = 0
        
        # Features d'EuroQol
        euroqol_vars = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales',
                       'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']
        
        for eq_var in euroqol_vars:
            dades_eq = dades_pacient[dades_pacient['Variable'] == eq_var]['valor_ordinal'].dropna()
            if len(dades_eq) > 0:
                features_pacient[f'{eq_var}_mitjana'] = dades_eq.mean()
                features_pacient[f'{eq_var}_primera'] = dades_eq.iloc[0]
                features_pacient[f'{eq_var}_disponible'] = 1
            else:
                features_pacient[f'{eq_var}_disponible'] = 0
        
        # Temps total de seguiment
        dates = pd.to_datetime(dades_pacient['Fecha']).dropna()
        if len(dates) > 1:
            features_pacient['temps_seguiment_dies'] = (dates.max() - dates.min()).days
        else:
            features_pacient['temps_seguiment_dies'] = 0
            
        # Nombre total de mesures
        features_pacient['total_mesures'] = len(dades_pacient)
        
        pacients_df.append(features_pacient)
    
    return pd.DataFrame(pacients_df)

# Crear dataset de pacients
features_pacients = crear_features_pacient_sense_imputacio(df_pandas)

# Variable objectiu
y = features_pacients['fallecido'].astype(int)

# Variables bàsiques i anàlisi de completitud
variables_cliniques = ['ALBUMINA', 'PCR', 'CREATININA', 'Rockwood']
euroqol_vars = ['EQ5D_Movilidad', 'EQ5D_CuidadoPersonal', 'EQ5D_ActividadesHabituales',
               'EQ5D_DolorMalestar', 'EQ5D_AnsiedadDepresion']

variables_base = ['temps_seguiment_dies', 'total_mesures']

# Variables categòriques 
features_categoriques = ['sexo', 'TIPO_PACIENTE_FUSIONADO', 'HOSPITAL_FUSIONADO', 'ZONA_FUSIONADA']

# Flags de disponibilitat
flags_disponibilitat = []
for var in variables_cliniques + euroqol_vars:
    flags_disponibilitat.append(f'{var}_disponible')

X_base = features_pacients[variables_base + flags_disponibilitat].copy()

# Afegir edat si està disponible 
X_base['edat_anys'] = features_pacients['edat_anys'].fillna(-1)
X_base['edat_disponible'] = (features_pacients['edat_anys'].notna()).astype(int)

# Processar variables categòriques
X_categoriques = pd.DataFrame()
label_encoders = {}

for cat_col in features_categoriques:
    if cat_col in features_pacients.columns:
        le = LabelEncoder()
        dades_cat = features_pacients[cat_col].fillna('Desconegut')
        X_categoriques[cat_col] = le.fit_transform(dades_cat)
        label_encoders[cat_col] = le

# Variables amb millor completitud
completitud = {}
for var in variables_cliniques:
    completitud[var] = features_pacients[f'{var}_disponible'].sum()

vars_completes = sorted(completitud.keys(), key=lambda x: completitud[x], reverse=True)
top_vars = vars_completes[:2]

# ESTRATÈGIA MENYS RESTRICTIVA: només exigir variables bàsiques + edat
mask_basic = features_pacients['edat_anys'].notna()
pacients_basics = features_pacients[mask_basic].copy()

# ESTRATÈGIA MODERADA: exigir almenys una variable clínica
mask_moderat = mask_basic & (
    (features_pacients['ALBUMINA_disponible'] == 1) |
    (features_pacients['PCR_disponible'] == 1) |
    (features_pacients['CREATININA_disponible'] == 1) |
    (features_pacients['Rockwood_disponible'] == 1)
)
pacients_moderats = features_pacients[mask_moderat].copy()

# Decidir quina estratègia utilitzar
if len(pacients_moderats) >= 50:
    pacients_complets = pacients_moderats.copy()
    estrategia_usada = "moderada"
    print(f" Usant estratègia moderada: {len(pacients_complets)} pacients")
elif len(pacients_basics) >= 50:
    pacients_complets = pacients_basics.copy()
    estrategia_usada = "bàsica"
    print(f" Usant estratègia bàsica: {len(pacients_complets)} pacients")
else:
    usar_casos_complets = False
    estrategia_usada = "només_flags"
    print(f"  Pocs casos disponibles. Usant només flags.")

usar_casos_complets = estrategia_usada != "només_flags"

if usar_casos_complets:
    print(f" Casos complets finals: {len(pacients_complets)}/{len(features_pacients)} ({len(pacients_complets)/len(features_pacients)*100:.1f}%)")
else:
    print("  Pocs casos complets nets. Usant estratègia amb flags.")

# Dataset 1: Tots els casos amb flags de disponibilitat
X_tots = pd.concat([X_base, X_categoriques], axis=1)

# Dataset 2: Casos amb estratègia adaptativa
if usar_casos_complets:
    features_complets_cols = ['edat_anys', 'temps_seguiment_dies', 'total_mesures']
    
    # Afegir variables clíniques disponibles
    for var in variables_cliniques:
        if f'{var}_disponible' in pacients_complets.columns:
            # Només afegir si almenys 20% dels pacients tenen aquesta variable
            disponibilitat = pacients_complets[f'{var}_disponible'].mean()
            if disponibilitat >= 0.2:
                features_complets_cols.extend([
                    f'{var}_primera', f'{var}_mitjana', f'{var}_max', 
                    f'{var}_min', f'{var}_n_mesures', f'{var}_tendencia'
                ])
    
    # Variables EuroQol disponibles
    for eq_var in euroqol_vars:
        if f'{eq_var}_disponible' in pacients_complets.columns:
            disponibilitat_eq = pacients_complets[f'{eq_var}_disponible'].mean()
            if disponibilitat_eq >= 0.1:  # Criteri més relaxat per EuroQol
                features_complets_cols.extend([f'{eq_var}_mitjana', f'{eq_var}_primera'])
    
    # Filtrar només les columnes que realment existeixen
    features_existents = []
    for col in features_complets_cols:
        if col in pacients_complets.columns:
            features_existents.append(col)
    
    print(f" Variables seleccionades: {len(features_existents)}")
    print(f" Variables: {features_existents}")  # Mostrar primeres 10
    
    X_complets_num = pacients_complets[features_existents].copy()
    
    # Variables categòriques incloses
    X_complets_cat = pd.DataFrame(index=pacients_complets.index)
    for cat_col in features_categoriques:
        if cat_col in pacients_complets.columns:
            dades_cat = pacients_complets[cat_col].fillna('Desconegut')
            X_complets_cat[cat_col] = label_encoders[cat_col].transform(dades_cat)
    
    X_complets = pd.concat([X_complets_num, X_complets_cat], axis=1)
    
    # Variable objectiu
    y_complet = pacients_complets['fallecido'].astype(int)
    
    # Eliminar files amb massa NaN

    # Contar NaN per fila
    nan_per_fila = X_complets.isnull().sum(axis=1)
    percentatge_nan = nan_per_fila / X_complets.shape[1]
    
    # Mantenir files amb menys del 50% de NaN
    mask_acceptable = percentatge_nan < 0.5
    
    X_complets_net = X_complets[mask_acceptable].copy()
    y_complet_net = y_complet[mask_acceptable].copy()
    
    # Per a les columnes restants amb NaN, omplir amb valors neutres
    # Variables numèriques: usar -999 com a indicador "sense dada"
    for col in X_complets_net.select_dtypes(include=[np.number]).columns:
        X_complets_net[col] = X_complets_net[col].fillna(-999)
    
    # Actualitzar variables finals
    X_complets = X_complets_net
    y_complet = y_complet_net
    
    print(f" Casos finals: {len(X_complets)}")
    print(f" Variables finals: {X_complets.iloc[0]}")
    print(f" NaN restants: {X_complets.isnull().sum().sum()}")
    
    # Verificar suficiència final
    if len(X_complets) < 30:
        print("  Molt pocs casos nets. Usant només estratègia amb flags.")
        usar_casos_complets = False

# Divisió train/test i entrenament de models
X_train_tots, X_test_tots, y_train_tots, y_test_tots = train_test_split(
    X_tots, y, test_size=0.2, random_state=42, stratify=y
)

# Escalat
scaler_tots = StandardScaler()
X_train_tots_scaled = scaler_tots.fit_transform(X_train_tots)
X_test_tots_scaled = scaler_tots.transform(X_test_tots)

# Models
models_tots = {
    'Random Forest': RandomForestClassifier(
        n_estimators=100, max_depth=8, min_samples_split=10, 
        min_samples_leaf=5, random_state=42, class_weight='balanced'
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100, max_depth=4, learning_rate=0.1, 
        random_state=42
    ),
    'Regressió Logística': LogisticRegression(
        random_state=42, class_weight='balanced', max_iter=1000
    )
}

# Entrenar models
resultats_tots = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for nom, model in models_tots.items():
    if nom in ['Regressió Logística']:
        model.fit(X_train_tots_scaled, y_train_tots)
        y_pred_proba = model.predict_proba(X_test_tots_scaled)[:, 1]
        cv_scores = cross_val_score(model, X_train_tots_scaled, y_train_tots, cv=cv, scoring='roc_auc')
    else:
        model.fit(X_train_tots, y_train_tots)
        y_pred_proba = model.predict_proba(X_test_tots)[:, 1]
        cv_scores = cross_val_score(model, X_train_tots, y_train_tots, cv=cv, scoring='roc_auc')
    
    auc_score = roc_auc_score(y_test_tots, y_pred_proba)
    
    resultats_tots[nom] = {
        'model': model,
        'auc_test': auc_score,
        'auc_cv_mean': cv_scores.mean(),
        'auc_cv_std': cv_scores.std(),
        'y_pred_proba': y_pred_proba
    }
    
    print(f"  {nom}: AUC {auc_score:.3f} (CV: {cv_scores.mean():.3f}±{cv_scores.std():.3f})")

# Modelatge amb casos complets 
resultats_complets = {}

if usar_casos_complets:
    X_train_comp, X_test_comp, y_train_comp, y_test_comp = train_test_split(
        X_complets, y_complet, test_size=0.2, random_state=42, stratify=y_complet
    )
    
    scaler_comp = StandardScaler()
    X_train_comp_scaled = scaler_comp.fit_transform(X_train_comp)
    X_test_comp_scaled = scaler_comp.transform(X_test_comp)
    
    for nom, _ in models_tots.items():
        if nom == 'Random Forest':
            model_comp = RandomForestClassifier(
                n_estimators=100, max_depth=8, min_samples_split=5, 
                min_samples_leaf=2, random_state=42, class_weight='balanced'
            )
        elif nom == 'Gradient Boosting':
            model_comp = GradientBoostingClassifier(
                n_estimators=100, max_depth=4, learning_rate=0.1, 
                random_state=42
            )
        else:
            model_comp = LogisticRegression(
                random_state=42, class_weight='balanced', max_iter=1000
            )
        
        if nom in ['Regressió Logística']:
            model_comp.fit(X_train_comp_scaled, y_train_comp)
            y_pred_proba_comp = model_comp.predict_proba(X_test_comp_scaled)[:, 1]
            try:
                cv_scores_comp = cross_val_score(model_comp, X_train_comp_scaled, y_train_comp, cv=cv, scoring='roc_auc')
            except:
                cv_scores_comp = np.array([np.nan])
        else:
            model_comp.fit(X_train_comp, y_train_comp)
            y_pred_proba_comp = model_comp.predict_proba(X_test_comp)[:, 1]
            try:
                cv_scores_comp = cross_val_score(model_comp, X_train_comp, y_train_comp, cv=cv, scoring='roc_auc')
            except:
                cv_scores_comp = np.array([np.nan])
        
        auc_score_comp = roc_auc_score(y_test_comp, y_pred_proba_comp)
        
        resultats_complets[nom] = {
            'model': model_comp,
            'auc_test': auc_score_comp,
            'auc_cv_mean': cv_scores_comp.mean(),
            'auc_cv_std': cv_scores_comp.std(),
            'y_pred_proba': y_pred_proba_comp
        }
        
        print(f"  {nom}: AUC {auc_score_comp:.3f} (CV: {cv_scores_comp.mean():.3f}±{cv_scores_comp.std():.3f})")

# Determinar millor model i estratègia
millor_tots = max(resultats_tots.keys(), key=lambda x: resultats_tots[x]['auc_test'])

if usar_casos_complets:
    millor_complets = max(resultats_complets.keys(), key=lambda x: resultats_complets[x]['auc_test'])
    
    if resultats_complets[millor_complets]['auc_test'] > resultats_tots[millor_tots]['auc_test']:
        print(f"\n MILLOR OPCIÓ: Casos complets ({millor_complets}) - AUC: {resultats_complets[millor_complets]['auc_test']:.3f}")
        millor_estrategia = 'complets'
        millor_model_final = resultats_complets[millor_complets]['model']
        millor_auc_final = resultats_complets[millor_complets]['auc_test']
    else:
        print(f"\n MILLOR OPCIÓ: Dataset complet amb flags ({millor_tots}) - AUC: {resultats_tots[millor_tots]['auc_test']:.3f}")
        millor_estrategia = 'tots'
        millor_model_final = resultats_tots[millor_tots]['model']
        millor_auc_final = resultats_tots[millor_tots]['auc_test']
else:
    print(f"\n MODEL FINAL: Dataset complet amb flags ({millor_tots}) - AUC: {resultats_tots[millor_tots]['auc_test']:.3f}")
    millor_estrategia = 'tots'
    millor_model_final = resultats_tots[millor_tots]['model']
    millor_auc_final = resultats_tots[millor_tots]['auc_test']

# VISUALITZACIÓ DE RESULTATS
# ============================================================================

plt.figure(figsize=(15, 10))

# Subplot 1: Comparació de models dataset complet
plt.subplot(2, 3, 1)

colors = ['red', 'yellow', 'green']

for i, (nom, resultat) in enumerate(resultats_tots.items()):
    fpr, tpr, _ = roc_curve(y_test_tots, resultat['y_pred_proba'])
    auc = resultat['auc_test']
    color = colors[i % len(colors)]  
    plt.plot(fpr, tpr, color=color, label=f'{nom} (AUC = {auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positius')
plt.ylabel('Taxa de Veritables Positius')
plt.title('ROC - Dataset Complet')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)


# Subplot 2: Comparació casos complets 
if usar_casos_complets:
    plt.subplot(2, 3, 2)
    for i, (nom, resultat) in enumerate(resultats_complets.items()):
        fpr, tpr, _ = roc_curve(y_test_comp, resultat['y_pred_proba'])
        auc = resultat['auc_test']
        color = colors[i % len(colors)]
        plt.plot(fpr, tpr, color=color, label=f'{nom} (AUC = {auc:.3f})', linewidth=2)
    
    plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positius')
    plt.ylabel('Taxa de Veritables Positius')
    plt.title('ROC - Casos Complets')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)

# Subplot 3: Completitud de dades
plt.subplot(2, 3, 3)
completitud_vars = []
completitud_vals = []

for var in variables_cliniques[:4]:
    completitud_vars.append(var)
    completitud_vals.append((features_pacients[f'{var}_disponible'].sum() / len(features_pacients)) * 100)

plt.barh(completitud_vars, completitud_vals, color='lightblue')
plt.xlabel('% Completitud')
plt.title('Completitud de Variables Clíniques')
plt.grid(True, alpha=0.3)

# Subplot 4: Importància de variables
plt.subplot(2, 3, 4)
if hasattr(millor_model_final, 'feature_importances_'):
    if millor_estrategia == 'tots':
        importancies = millor_model_final.feature_importances_
        noms_features = X_train_tots.columns
    else:
        importancies = millor_model_final.feature_importances_
        noms_features = X_train_comp.columns
    
    df_importancia = pd.DataFrame({
        'variable': noms_features,
        'importancia': importancies
    }).sort_values('importancia', ascending=False).head(20)
    
    plt.barh(range(len(df_importancia)), df_importancia['importancia'], color='red')
    plt.yticks(range(len(df_importancia)), df_importancia['variable'])
    plt.xlabel('Importància')
    plt.title(f'Top Variables - {millor_tots if millor_estrategia == "tots" else millor_complets}')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3)

# Subplot 5: Distribució de probabilitats predites
plt.subplot(2, 3, 5)
if millor_estrategia == 'tots':
    y_pred_millor = resultats_tots[millor_tots]['y_pred_proba']
    y_test_millor = y_test_tots
else:
    y_pred_millor = resultats_complets[millor_complets]['y_pred_proba']
    y_test_millor = y_test_comp

plt.hist(y_pred_millor[y_test_millor == 0], bins=20, alpha=0.7, label='Supervivents', color='blue')
plt.hist(y_pred_millor[y_test_millor == 1], bins=20, alpha=0.7, label='Difunts', color='red')
plt.xlabel('Probabilitat Predita')
plt.ylabel('Freqüència')
plt.title('Distribució de Probabilitats')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 6: Matriu de confusió
plt.subplot(2, 3, 6)
fpr_millor, tpr_millor, thresholds = roc_curve(y_test_millor, y_pred_millor)
optimal_idx = np.argmax(tpr_millor - fpr_millor)
optimal_threshold = thresholds[optimal_idx]

y_pred_binary = (y_pred_millor >= optimal_threshold).astype(int)
cm = confusion_matrix(y_test_millor, y_pred_binary)

sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['Supervivents', 'Difunts'],
            yticklabels=['Supervivents', 'Difunts'])
plt.title(f'Matriu de Confusió\n(Llindar = {optimal_threshold:.3f})')
plt.ylabel('Valor Real')
plt.xlabel('Predicció')

plt.tight_layout()
plt.show()

# Descompondre la matriu
TN, FP, FN, TP = cm.ravel()

# Mètriques de validació
accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)  # Recall o True Positive Rate
specificity = TN / (TN + FP)  # True Negative Rate
precision = TP / (TP + FP) if (TP + FP) != 0 else 0
f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) != 0 else 0

# Mostrar els resultats
print(f"Accuracy:     {accuracy:.3f}")
print(f"Sensibilitat: {sensitivity:.3f}")
print(f"Especificitat:{specificity:.3f}")
print(f"Precisión:    {precision:.3f}")
print(f"F1-Score:     {f1:.3f}")

# CONCLUSIONS
# ============================================================================

print(f"\n CONCLUSIONS :")
print("="*50)
print(f" Millor estratègia: {'Casos complets' if millor_estrategia == 'complets' else 'Dataset amb flags'}")
print(f" Millor model: {millor_complets if millor_estrategia == 'complets' else millor_tots}")
print(f" AUC final: {millor_auc_final:.3f}")
print(f" Total pacients: {len(features_pacients):,}")
print(f" Taxa mortalitat: {features_pacients['fallecido'].mean():.1%}")

if hasattr(millor_model_final, 'feature_importances_'):
    print(f"\n TOP 5 VARIABLES MÉS IMPORTANTS:")
    if millor_estrategia == 'tots':
        noms_features = X_train_tots.columns
    else:
        noms_features = X_train_comp.columns
    
    df_importancia = pd.DataFrame({
        'variable': noms_features,
        'importancia': millor_model_final.feature_importances_
    }).sort_values('importancia', ascending=False)
    
    for i, row in df_importancia.head(5).iterrows():
        print(f"  {row['variable']}: {row['importancia']:.4f}")

# CROSS-VALIDATION
# ============================================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_validate, learning_curve
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

def cross_validation_complet(models_dict, X, y, cv_folds=5, random_state=42):
    """
    Realitza cross-validation complet amb múltiples mètriques
    """
    
    # Definir mètriques d'avaluació
    scoring = {
        'roc_auc': 'roc_auc',
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1'
    }
    
    # Configurar cross-validation
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=random_state)
    
    resultats_cv = {}
    
    for nom_model, model in models_dict.items():
        print(f"\n Avaluant {nom_model}...")
        
        # Executar cross-validation
        cv_results = cross_validate(
            model, X, y, 
            cv=cv, 
            scoring=scoring,
            return_train_score=True,
            n_jobs=-1
        )
        
        # Calcular estadístiques
        resultats_model = {}
        for metric in scoring.keys():
            test_scores = cv_results[f'test_{metric}']
            train_scores = cv_results[f'train_{metric}']
            
            resultats_model[f'{metric}_test_mean'] = np.mean(test_scores)
            resultats_model[f'{metric}_test_std'] = np.std(test_scores)
            resultats_model[f'{metric}_train_mean'] = np.mean(train_scores)
            resultats_model[f'{metric}_train_std'] = np.std(train_scores)
            resultats_model[f'{metric}_test_scores'] = test_scores
            resultats_model[f'{metric}_train_scores'] = train_scores
            
            # Calcular overfitting 
            overfitting = np.mean(train_scores) - np.mean(test_scores)
            resultats_model[f'{metric}_overfitting'] = overfitting
        
        resultats_cv[nom_model] = resultats_model
        
        # Mostrar resultats
        print(f"  AUC Test:  {resultats_model['roc_auc_test_mean']:.3f} ± {resultats_model['roc_auc_test_std']:.3f}")
        print(f"  AUC Train: {resultats_model['roc_auc_train_mean']:.3f} ± {resultats_model['roc_auc_train_std']:.3f}")
        print(f"  Overfitting: {resultats_model['roc_auc_overfitting']:.3f}")
        print(f"  Accuracy:  {resultats_model['accuracy_test_mean']:.3f} ± {resultats_model['accuracy_test_std']:.3f}")
        print(f"  F1-Score:  {resultats_model['f1_test_mean']:.3f} ± {resultats_model['f1_test_std']:.3f}")
    
    return resultats_cv

def visualitzar_cv_results(resultats_cv):
    """
    Crea visualitzacions dels resultats de cross-validation
    """
    
    # Preparar dades per visualització
    models = list(resultats_cv.keys())
    metriques = ['roc_auc', 'accuracy', 'precision', 'recall', 'f1']
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Resultats Cross-Validation', fontsize=16, fontweight='bold')
    
    # 1. Comparació mètriques principals
    ax1 = axes[0, 0]
    x_pos = np.arange(len(models))
    width = 0.15
    
    colors=['pink', 'orange', 'purple']

    for i, metric in enumerate(['roc_auc', 'accuracy', 'f1']):
        means = [resultats_cv[model][f'{metric}_test_mean'] for model in models]
        stds = [resultats_cv[model][f'{metric}_test_std'] for model in models]
        
        ax1.bar(x_pos + i*width, means, width, 
               yerr=stds, capsize=3, 
               label=metric.replace('_', ' ').title(), color = colors[i % len(colors)],
               alpha=0.8)
    
    ax1.set_xlabel('Models')
    ax1.set_ylabel('Puntuació')
    ax1.set_title('Comparació Mètriques Principals')
    ax1.set_xticks(x_pos + width)
    ax1.set_xticklabels(models, rotation=45)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Boxplot AUC per model
    ax2 = axes[0, 1]
    auc_data = []
    labels_auc = []
    
    for model in models:
        auc_scores = resultats_cv[model]['roc_auc_test_scores']
        auc_data.append(auc_scores)
        labels_auc.append(model)
    
    box_plot = ax2.boxplot(auc_data, labels=labels_auc, patch_artist=True)
    colors = ['red', 'yellow', 'green']
    for patch, color in zip(box_plot['boxes'], colors):
        patch.set_facecolor(color)
    
    ax2.set_title('Distribució AUC Cross-Validation')
    ax2.set_ylabel('AUC Score')
    ax2.grid(True, alpha=0.3)
    plt.setp(ax2.get_xticklabels(), rotation=45)
    
    # 3. Overfitting comparison
    ax3 = axes[0, 2]
    overfitting_auc = [resultats_cv[model]['roc_auc_overfitting'] for model in models]
    overfitting_acc = [resultats_cv[model]['accuracy_overfitting'] for model in models]
    
    x_pos = np.arange(len(models))
    width = 0.35
    
    ax3.bar(x_pos - width/2, overfitting_auc, width, label='AUC', alpha=0.8, color='red')
    ax3.bar(x_pos + width/2, overfitting_acc, width, label='Accuracy', alpha=0.8, color='blue')
    
    ax3.set_xlabel('Models')
    ax3.set_ylabel('Overfitting (Train - Test)')
    ax3.set_title('Nivell d\'Overfitting per Model')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(models, rotation=45)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    # 4. Matriu de correlació entre mètriques
    ax4 = axes[1, 0]
    
    # Crear dataframe amb totes les mètriques
    df_metriques = pd.DataFrame()
    for model in models:
        for metric in metriques:
            scores = resultats_cv[model][f'{metric}_test_scores']
            for i, score in enumerate(scores):
                df_metriques = pd.concat([df_metriques, pd.DataFrame({
                    'Model': [model],
                    'Fold': [i],
                    'AUC': [resultats_cv[model]['roc_auc_test_scores'][i]],
                    'Accuracy': [resultats_cv[model]['accuracy_test_scores'][i]],
                    'Precision': [resultats_cv[model]['precision_test_scores'][i]],
                    'Recall': [resultats_cv[model]['recall_test_scores'][i]],
                    'F1': [resultats_cv[model]['f1_test_scores'][i]]
                })], ignore_index=True)
    
    corr_matrix = df_metriques[['AUC', 'Accuracy', 'Precision', 'Recall', 'F1']].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, ax=ax4)
    ax4.set_title('Correlació entre Mètriques')
    
    # 5. Estabilitat dels models
    ax5 = axes[1, 1]
    
    variabilitat = {}
    for model in models:
        variabilitat[model] = resultats_cv[model]['roc_auc_test_std']
    
    models_sorted = sorted(variabilitat.keys(), key=lambda x: variabilitat[x])
    vals = [variabilitat[model] for model in models_sorted]
    
    bars = ax5.bar(models_sorted, vals, color=['red', 'green', 'yellow'], alpha=0.7)
    ax5.set_xlabel('Models')
    ax5.set_ylabel('Desviació Estàndard AUC')
    ax5.set_title('Estabilitat dels Models')
    ax5.grid(True, alpha=0.3)
    plt.setp(ax5.get_xticklabels(), rotation=45)
    
    # Afegir valors a les barres
    for bar, val in zip(bars, vals):
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                f'{val:.3f}', ha='center', va='bottom')
    
    # 6. Rendiment per fold
    ax6 = axes[1, 2]
    
    colors = ['red', 'yellow', 'green']  

    for i, model in enumerate(models):
        folds = range(1, len(resultats_cv[model]['roc_auc_test_scores']) + 1)
        auc_scores = resultats_cv[model]['roc_auc_test_scores']
        ax6.plot(folds, auc_scores, marker='o', color=colors[i], label=model, linewidth=2)
    
    ax6.set_xlabel('Fold')
    ax6.set_ylabel('AUC Score')
    ax6.set_title('Rendiment per Fold')
    ax6.legend()
    ax6.grid(True, alpha=0.3)
    ax6.set_xticks(range(1, 6))
    
    plt.tight_layout()
    plt.show()

def crear_taula_resum_cv(resultats_cv):
    """
    Crea una taula resum dels resultats de cross-validation
    """
    
    resum_data = []
    
    for model in resultats_cv.keys():
        fila = {
            'Model': model,
            'AUC_Mean': f"{resultats_cv[model]['roc_auc_test_mean']:.3f}",
            'AUC_Std': f"{resultats_cv[model]['roc_auc_test_std']:.3f}",
            'Accuracy_Mean': f"{resultats_cv[model]['accuracy_test_mean']:.3f}",
            'Accuracy_Std': f"{resultats_cv[model]['accuracy_test_std']:.3f}",
            'F1_Mean': f"{resultats_cv[model]['f1_test_mean']:.3f}",
            'F1_Std': f"{resultats_cv[model]['f1_test_std']:.3f}",
            'Overfitting_AUC': f"{resultats_cv[model]['roc_auc_overfitting']:.3f}",
            'Estabilitat': 'Alta' if resultats_cv[model]['roc_auc_test_std'] < 0.05 else 'Baixa'
        }
        resum_data.append(fila)
    
    df_resum = pd.DataFrame(resum_data)
    
    print("\n TAULA RESUM CROSS-VALIDATION")
    print("="*80)
    print(df_resum.to_string(index=False))
    
    return df_resum

# Per dataset amb flags
resultats_cv_tots = cross_validation_complet(models_tots, X_train_tots, y_train_tots)

# Per casos complets 
if usar_casos_complets:
    resultats_cv_complets = cross_validation_complet(models_tots, X_train_comp, y_train_comp)
    
    print(f"\n COMPARACIÓ ESTRATÈGIES:")
    print("="*50)
    
    for model in models_tots.keys():
        auc_tots = resultats_cv_tots[model]['roc_auc_test_mean']
        auc_comp = resultats_cv_complets[model]['roc_auc_test_mean']
        
        print(f"{model}:")
        print(f"  Dataset complet: {auc_tots:.3f}")
        print(f"  Casos complets:  {auc_comp:.3f}")
        print(f"  Diferència:      {auc_comp - auc_tots:+.3f}")

# Crear visualitzacions
visualitzar_cv_results(resultats_cv_tots)

if usar_casos_complets:
    visualitzar_cv_results(resultats_cv_complets)

# Crear taules 
print("\n" + "="*80)
print("DATASET COMPLET AMB FLAGS:")
taula_tots = crear_taula_resum_cv(resultats_cv_tots)

if usar_casos_complets:
    print("\n" + "="*80)
    print("CASOS COMPLETS:")
    taula_complets = crear_taula_resum_cv(resultats_cv_complets)

# Determinar millor model amb cross-validation
millor_model_cv = max(resultats_cv_tots.keys(), 
                     key=lambda x: resultats_cv_tots[x]['roc_auc_test_mean'])

print(f"\n MILLOR MODEL SEGONS CROSS-VALIDATION:")
print(f"  Model: {millor_model_cv}")
print(f"  AUC CV: {resultats_cv_tots[millor_model_cv]['roc_auc_test_mean']:.3f} ± {resultats_cv_tots[millor_model_cv]['roc_auc_test_std']:.3f}")
print(f"  Overfitting: {resultats_cv_tots[millor_model_cv]['roc_auc_overfitting']:.3f}")
print(f"  Estabilitat: {'Alta' if resultats_cv_tots[millor_model_cv]['roc_auc_test_std'] < 0.05 else 'Baixa'}")
